<a href="https://colab.research.google.com/github/KevinUnedo/machine-learning-clustering/blob/main/Group11_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Taking user input for two numbers
num1 = float(input("Enter the first number: "))
num2 = float(input("Enter the second number: "))

# Adding the numbers
sum_result = num1 + num2

# Displaying the result
print("The sum of", num1, "and", num2, "is:", sum_result)


Enter the first number: 1
Enter the second number: 2
The sum of 1.0 and 2.0 is: 3.0
